In [26]:
import numpy as np
import os

In [27]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))
    x_test = np.load(os.path.join(folder, 'x_test.npy'))
    y_test = np.load(os.path.join(folder, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [28]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        rng = np.random.default_rng(seed=0)
        self.w = rng.normal(size=(dim, 1)) / np.sqrt(dim)
        self.b = np.zeros((1,))

    def predict(self, x, probs=False):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"

        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')

    def fit(self, x, y, iters=1000, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        assert len(x) == len(y), \
            "Количество экземпляров в массиве X не равно количеству меток в массиве Y. " + \
            f"Полученные размеры: len(X) = {len(x)}, len(Y) = {len(y)}."
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
        return self

## 1. Применение логистической регрессии (несбалансированные данные)


### 1.1 Создание и обучение логистической регрессии


In [29]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')

In [30]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.
lr = LogisticRegression(x_train.shape[1])
lr.fit(x_train, y_train, iters=10_000)

In [31]:
# Получите предсказания на тестовой выборке и оцените точность модели,
# используя accuracy_score из пакета SciKit-Learn.
from sklearn.metrics import accuracy_score
accuracy_score(
    y_test,
    lr.predict(x_test)
)

0.9318181818181818

### 1.2 Анализ качества модели


In [32]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`.

class DummyClassifier:
    def __init__(self, dt=np.float32):
        self.dt = dt

    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Должен возвращаться массив N предсказаний
        return np.zeros(x.shape[0], dtype=self.dt)

In [33]:
# Оцените точность "глупого классификатора", объясните результат.
dc = DummyClassifier()
assert x_test.shape[0] == dc.predict(x_test).shape[0]

In [34]:
# Используйте дополнительные метрики (f1-score, recall, precision) из пакета sklearn для анализа "глупого классификатора".
from sklearn.metrics import (
    f1_score,
    recall_score,
    precision_score
)


def get_stats(model, x_test, y_test, avg='binary') -> tuple:
    m = model
    y_pred = m.predict(x_test).astype(np.bool)
    f1 = f1_score(y_test, y_pred, average=avg)
    recall = recall_score(y_test, y_pred, average=avg)
    precision = precision_score(y_test, y_pred, average=avg)
    print(f'{f1=}')
    print(f'{recall=}')
    print(f'{precision=}')

In [35]:
get_stats(dc, x_test, y_test)

f1=np.float64(0.0)
recall=np.float64(0.0)
precision=np.float64(0.0)


/home/nia/.pyenv/versions/3.10.14/envs/ssau3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
# Используя те же метрики, проанализируйте обученную вами модель логистической регрессии.
get_stats(lr, x_test, y_test)

f1=np.float64(0.6341463414634146)
recall=np.float64(0.65)
precision=np.float64(0.6190476190476191)


In [37]:
# Объясните результат, описав его комментариями в этой клетке.
total_ones = 0

dc_pred = dc.predict(x_test)
dc_ones = 0
dc_correct_ones = 0
dc_recall_ones = 0

lr_pred = lr.predict(x_test)
lr_ones = 0
lr_correct_ones = 0
lr_recall_ones = 0

for dc_y_pred, lr_y_pred, y_real in zip(dc_pred, lr_pred, y_test):
    if dc_y_pred == 1:
        dc_ones += 1
        if dc_y_pred == y_real:
            dc_correct_ones += 1
    if lr_y_pred == 1:
        lr_ones += 1
        if lr_y_pred == y_real:
            lr_correct_ones += 1
    if y_real == 1:
        total_ones += 1
        if dc_y_pred == y_real:
            dc_recall_ones += 1
        if lr_y_pred == y_real:
            lr_recall_ones += 1

dc_precision = 0
lr_precision = lr_correct_ones/lr_ones

dc_recall = 0
lr_recall = lr_recall_ones/total_ones

print(
    f"""
# Общие термины
Глупый классификатор - DC
Корректный классификатор - LR


# Precision

DC классификатор нашёл {dc_ones} единиц
LR классификатор нашёл {lr_ones} единиц

Из которых правильно предсказанные:
Для DC: {dc_correct_ones}
Для LR: {lr_correct_ones}

Соответственно метрика Precision:
DC: dc_correct_ones/dc_ones=0
LR: {lr_correct_ones/lr_ones=}


# Recall

Всего меток с единицей {total_ones}

Из которых правильно предсказанные:
Для DC: {dc_recall_ones}
Для LR: {lr_recall_ones}

Соответственно метрика Recall:
DC: {dc_recall_ones/total_ones=}
LR: {lr_recall_ones/total_ones=}


# F1 
DC: (2 * dc_precision * dc_recall) / (dc_precision + dc_recall)=0
LR: {(2 * lr_precision * lr_recall) / (lr_precision + lr_recall)=}


# Общий вывод
Так как вышеописанные метрики расчитываются относительно единичного класса,
то для DC классификатора метрики равны нулю, так как модель предсказывает всегда ноль
"""
)


# Общие термины
Глупый классификатор - DC
Корректный классификатор - LR


# Precision

DC классификатор нашёл 0 единиц
LR классификатор нашёл 21 единиц

Из которых правильно предсказанные:
Для DC: 0
Для LR: 13

Соответственно метрика Precision:
DC: dc_correct_ones/dc_ones=0
LR: lr_correct_ones/lr_ones=0.6190476190476191


# Recall

Всего меток с единицей 20

Из которых правильно предсказанные:
Для DC: 0
Для LR: 13

Соответственно метрика Recall:
DC: dc_recall_ones/total_ones=0.0
LR: lr_recall_ones/total_ones=0.65


# F1 
DC: (2 * dc_precision * dc_recall) / (dc_precision + dc_recall)=0
LR: (2 * lr_precision * lr_recall) / (lr_precision + lr_recall)=0.6341463414634146


# Общий вывод
Так как вышеописанные метрики расчитываются относительно единичного класса,
то для DC классификатора метрики равны нулю, так как модель предсказывает всегда ноль



### 1.3 Анализ набора данных


In [38]:
# Посчитайте количество экземпляров данных для каждого класса.
def count_classes(y_test):
    total_count = {}
    for y in set(y_test):
        total_count[y] = len(y_test[np.where(
            y_test == y
        )])
    return total_count


count_classes(y_test)

{np.float32(0.0): 200, np.float32(1.0): 20}

In [39]:
# Предложите способ улучшения качества модели.
# Подсказка: добавление дубликатов в данные.
# Указание: не изменяйте тестовую выборку.
def find_best_train(
    x_train: np.ndarray,
    y_train: np.ndarray,
    dups_of_class: int = 0,
    shuffle: bool = False,
    seed: int = None
) -> tuple[np.array, np.array]:
    """Досоздание данных для обучения

    Args:
        x_train (np.array размерности [N, dim]): 
            Массив входных признаков.
        y_train (np.array размернсоти [N]): 
            Массив меток (правильных ответов).
        shuffle (bool, optional): 
            Перемешивать данные или нет. 
            Defaults to False.
        seed (int, optional): 
            Сид для перемешивания данных. 
            Defaults to None.
        dups_of_class (int, optional): 
            Колличество дубликатов на каждый класс. 
            Defaults to 0.

    Returns:
        tuple[np.array, np.array]:
        Сбаланисированные массивы входных признаков и меток
    """

    def get_intervals(ind):
        return ind[0][0], ind[0][-1], len(ind[0])

    if dups_of_class < 0:
        raise Exception(
            'Параметр дубликатов датасета должен быть не отрицательным числом')

    np.random.seed(seed)

    xt, yt = x_train.copy(), y_train.copy()

    total_count = count_classes(yt)
    cnt = list(total_count.values())

    ones_indicies = np.where(yt == 1)

    low, high, _ = get_intervals(ones_indicies)
    poses = np.random.randint(low=low, high=high, size=cnt[0] - cnt[1])

    xt = np.vstack((xt, xt[poses]))
    yt = np.hstack((yt, yt[poses]))

    assert xt.shape[0] == yt.shape[0]

    if dups_of_class:
        ind = np.where(yt == 0)
        low, high, _ = get_intervals(ind)
        poses = np.random.randint(low=low, high=high, size=dups_of_class)
        xt = np.vstack((xt[poses], xt))
        yt = np.hstack((yt[poses], yt))
        
        ind = np.where(yt == 1)
        low, high, _ = get_intervals(ind)
        poses = np.random.randint(low=low, high=high, size=dups_of_class)
        xt = np.vstack((xt, xt[poses]))
        yt = np.hstack((yt, yt[poses]))

    if shuffle:
        idxs = np.arange(len(yt))
        np.random.shuffle(idxs)
        return xt[idxs], yt[idxs]
    return xt, yt

In [42]:
# Создайте и обучите модель с использованием предложенных наработок.
xt, yt = find_best_train(
    x_train,
    y_train,
    dups_of_class=0,
    shuffle=False,
    seed=42424
)
bm = LogisticRegression(x_train.shape[1])
bm.fit(xt, yt)


# Оцените качество новой модели, используя метрики из пакета sklearn.metrics.
# Указание: постарайтесь сбалансировать данные таким образом, чтобы новая модель была ощутимо лучше старой.
print('Old model')
get_stats(lr, x_test, y_test)
print('------ ------')
print('New model')
get_stats(bm, x_test, y_test)

Old model
f1=np.float64(0.6341463414634146)
recall=np.float64(0.65)
precision=np.float64(0.6190476190476191)
------ ------
New model
f1=np.float64(0.7441860465116279)
recall=np.float64(0.8)
precision=np.float64(0.6956521739130435)


## 2. Применение логистической регрессии (нелинейные данные)


In [17]:
x_train, y_train, x_test, y_test = load_data('dataset2')

In [18]:
# Создайте и обучите модель но на этом наборе данных.
lr = LogisticRegression(x_train.shape[1])
lr.fit(x_train, y_train)

In [19]:
# Проанализируйте качество модели.
get_stats(lr, x_test, y_test)

f1=np.float64(0.6194690265486725)
recall=np.float64(0.7777777777777778)
precision=np.float64(0.5147058823529411)


In [20]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, ...).
# Объедините трансформированные данные с исходными (важно: количество экземпляров в x_train не должно увеличиться).
x_train, y_train, x_test, y_test = load_data('dataset2')


OPS = [
    np.sin,
    np.cos,
    np.tan,
]


def apply_ops(
    x_train: np.ndarray,
    ops: list[np.ufunc]
) -> np.ndarray:
    xt = x_train.copy()
    input_size = xt.shape[0]
    for op in ops:
        assert type(op) is np.ufunc
        xt = np.hstack(
            (xt, op(xt))
        )
    assert input_size == xt.shape[0]
    return xt


x_train = apply_ops(x_train, OPS)

In [21]:
# Создайте и обучите модель с использованием наработок.
lr = LogisticRegression(x_train.shape[1])
lr.fit(x_train, y_train)

In [22]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics.
# Указание: постарайтесь добиться точности в 100%!
get_stats(lr, apply_ops(x_test, OPS), y_test)

f1=np.float64(1.0)
recall=np.float64(1.0)
precision=np.float64(1.0)


## 3. Доп. задания (любое на выбор, опционально)


### 3.2 'Обобщение' логистической регрессии


Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.


In [23]:
x_train, y_train, x_test, y_test = load_data('dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.


In [24]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий, 
где `C` - количество классов. i-ая логистическая регрессия производит 
бинарную классификацию вида: все остальные классы и i-ый класс.
"""


class MulticlassLogisticRegression:

    def __init__(self, n_classes, dim):
        self.regressions: dict[int, LogisticRegression] = {}
        for c in range(n_classes):
            self.regressions[c] = LogisticRegression(dim)
        self.default_predict_class = 0

    def predict(
        self,
        x: np.ndarray
    ) -> np.ndarray:
        """
        Args:
            x (np.ndarray) [N, dim]:
                Массив входных признаков.

        Returns:
            np.ndarray:
                Возвращается массив целых чисел размерности [N],
                где i-ый элемент обозначает номер класса для
                i-го экземпляра данных в `x`.
        """
        data = []
        for model in self.regressions.values():
            data.append(model.predict(x, probs=True))

        result = np.zeros(x.shape[0])
        for r_idx in range(x.shape[0]):
            decided_class = self.default_predict_class
            max_prob = 0
            for model_idx, model_value in enumerate(self.regressions.keys()):
                if data[model_idx][r_idx][0] >= max_prob:
                    decided_class = model_value
                    max_prob = data[model_idx][r_idx][0]
            result[r_idx] = decided_class
        return result.astype('int32')

    def fit(
        self,
        x: np.ndarray,
        y: np.ndarray
    ):
        for c in set(y):
            if c not in self.regressions.keys():
                raise Exception('Неизвестный класс')
        for c in self.regressions.keys():
            prepared_y = np.zeros_like(y)
            cls = np.where(
                c == y
            )
            prepared_y[cls] = 1
            plr = self.regressions[c]
            plr.fit(x, prepared_y)

In [25]:
# Создайте и обучите написанный классификатор.
from sklearn.metrics import classification_report

x_train, y_train, x_test, y_test = load_data('dataset3')
mlr = MulticlassLogisticRegression(
    n_classes=len(set(y_train)),
    dim=x_train.shape[1]
)
mlr.fit(x_train, y_train)
y_predicted = mlr.predict(x_test)

# Оцените точность модели.
LABELS = list(set(y_train))
print(classification_report(y_test, y_predicted, labels=LABELS))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        48
           1       0.90      0.97      0.94        39
           2       0.98      1.00      0.99        52
           3       0.94      0.96      0.95        49
           4       1.00      0.90      0.95        39
           5       0.88      1.00      0.93        50
           6       0.93      0.98      0.95        42
           7       0.89      0.98      0.93        43
           8       0.92      0.59      0.72        37
           9       0.92      0.92      0.92        51

    accuracy                           0.94       450
   macro avg       0.94      0.93      0.93       450
weighted avg       0.94      0.94      0.93       450

